# Loan Default Prediction
## Team Members:
* Harish Puvvada
* Vamsi Mohan Ramineedi

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#df2007_11 = pd.read_csv(os.getenv('FDS')+'LoanStats_2007_to_2011.csv',low_memory=False,skiprows=1)
df2012_13 = pd.read_csv(os.getenv('FDS')+'LoanStats_2012_to_2013.csv',low_memory=False,skiprows=1)
df2014 = pd.read_csv(os.getenv('FDS')+'LoanStats_2014.csv',low_memory=False,skiprows=1)

In [3]:
dataset = pd.concat([df2012_13, df2014]) #merging 2007 to 2014 datasets
dataset = dataset.iloc[:,2:111] #removing empty columns
empty_cols = [i for i in range(45,72)] #more empty columns
dataset = dataset.drop(dataset.columns[empty_cols],axis=1)
data_with_loanstatus_sliced = dataset[(dataset['loan_status']=="Fully Paid") | (dataset['loan_status']=="Charged Off")]
di = {"Fully Paid":0, "Charged Off":1} #converting target variable to boolean
Dataset_withBoolTarget= data_with_loanstatus_sliced.replace({"loan_status": di})

In [4]:
Dataset_withBoolTarget['loan_status'].value_counts()
Dataset_withBoolTarget.shape

(376233, 82)

In [5]:
#print(Dataset_withBoolTarget.shape)
dataset=Dataset_withBoolTarget.dropna(thresh = 340000,axis=1) #340000 is minimum number of non-NA values
#print(x.isnull().sum()) #- to check how many null values in all the columns
print(dataset.shape)
#dataset['loan_status'].value_counts()

(376233, 74)


In [6]:
del_col_names = ["delinq_2yrs",  "last_pymnt_d", "chargeoff_within_12_mths","delinq_amnt","emp_title", "term", "emp_title", "pymnt_plan","purpose","title", "zip_code", "verification_status", "dti","earliest_cr_line", "initial_list_status", "out_prncp",
"pymnt_plan", "num_tl_90g_dpd_24m", "num_tl_30dpd", "num_tl_120dpd_2m", "num_accts_ever_120_pd", "delinq_amnt", 
"chargeoff_within_12_mths", "total_rec_late_fee", "out_prncp_inv", "issue_d"] #deleting some more columns
dataset = dataset.drop(labels = del_col_names, axis = 1) 
dataset.shape


(376233, 52)

In [19]:
features = ['funded_amnt','emp_length','annual_inc','home_ownership','tax_liens','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] #selecting final features #'addr_state',
Final_data = dataset[features] #19 features with target var
Final_data["int_rate"] = Final_data["int_rate"].apply(lambda x:float(x[:-1]) ) #reomving % sign, conv to float  - int_rate column
Final_data["int_rate"].corr(Final_data["loan_status"])

0.24126296486971305

In [20]:
HTML(Final_data.tail().to_html())

,funded_amnt,emp_length,annual_inc,home_ownership,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
235623,15000.0,4 years,66000.0,RENT,0.0,B,509.12,0.0,0.0,13.53,8.0,6.0,50.0,120.0,96.5,580.0,2877.0,1.0,0
235624,18400.0,4 years,110000.0,MORTGAGE,0.0,C,7394.95,4.0,0.0,14.47,18.0,7.0,15.0,128.0,89.6,2203.0,17353.0,5.0,0
235625,22000.0,10+ years,78000.0,MORTGAGE,0.0,D,17.50,5.0,1.0,19.97,18.0,5.0,17.0,198.0,54.4,5569.0,13864.0,5.0,1
235626,2000.0,3 years,83000.0,OWN,0.0,A,1500.68,2.0,0.0,7.90,21.0,3.0,5.0,130.0,23.4,36402.0,28172.0,7.0,0
235628,10000.0,10+ years,46000.0,MORTGAGE,0.0,D,367.20,2.0,0.0,19.20,6.0,2.0,4.0,166.0,83.9,2175.0,11495.0,4.0,0


In [21]:
Final_data['grade'] = Final_data['grade'].map({'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1})
Final_data["home_ownership"] = Final_data["home_ownership"].map({"MORTGAGE":6,"RENT":5,"OWN":4,"OTHER":3,"NONE":2,"ANY":1})

In [24]:
Final_data["emp_length"] = Final_data["emp_length"].replace({'years':'','year':'',' ':'','<':'','\+':'','n/a':'0'}, regex = True)
Final_data.head()

,funded_amnt,emp_length,annual_inc,home_ownership,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,27050.0,10,55000.0,4,0.0,6,6074.19,4.0,0.0,10.99,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
1,12000.0,4,60000.0,5,0.0,6,3775.55,0.0,0.0,10.99,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0
2,4800.0,2,39600.0,6,0.0,6,3900.48,0.0,0.0,10.99,3.0,2.0,25.0,220.0,16.1,21564.0,1379.0,0.0,0
3,12000.0,10,130000.0,6,0.0,6,398.28,3.0,0.0,11.99,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
4,7550.0,3,28000.0,5,0.0,5,529.67,0.0,0.0,16.24,4.0,4.0,17.0,38.0,96.0,160.0,1440.0,1.0,0


In [23]:
Final_data["emp_length"].value_counts()

10+ years    122222
2 years       32828
3 years       28995
< 1 year      28121
5 years       24756
1 year        23477
4 years       21825
7 years       21701
6 years       21467
8 years       18453
n/a           17744
9 years       14644
Name: emp_length, dtype: int64

In [25]:
Final_data["emp_length"].value_counts()

10    122222
1      51598
2      32828
3      28995
5      24756
4      21825
7      21701
6      21467
8      18453
0      17744
9      14644
Name: emp_length, dtype: int64

In [15]:
#dataset[(dataset['avg_cur_bal'] < 5000 ) | (dataset['loan_status']==1)]

In [16]:
features = []

In [17]:
mandatory  - "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl","mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","avg_cur_bal","acc_open_past_24mths"

NameError: name 'mandatory' is not defined

In [ ]:
doubtful - acc_now_delinq, pct_tl_nvr_dlq, num_sats, sub_grade